# Query Data

How to query the processed data.

**config.yaml**

The `config.yaml` file contains the root and directories of each processed dataset.

**Sequence Data**

Sequence data takes the form:

```yaml
data: {
    'stimCode':
        (experiment-key): [action data]
    'respCode':
        (experiment-key): [response data]
}
```

where `experiment-key = (Rat ID, Session Type, Session Number)`

For example:

```python
{
    (4151, 'IDS_BL_Easy', 1): {
        'stimCode': array([0,1,...,0]),
        'respCode': array([0,1,...,0])
    },
    (4151, 'IDS_BL_Easy', 2): {
        'stimCode': array([0,1,...,0]),
        'respCode': array([0,1,...,0])
    },
    ...
}
```

---
```
: 22.06.24
: zachcolinwolpe@gmail.com
```
---

In [1]:
import pandas as pd
import numpy as np
import scipy.io
import logging
import pprint
import os

from process_data.extract_matlab_data import load_processed_data
from src.helpers import log_rat_metadata, log_sequence_data, load_config
from src.query_dataset import (QuerySequenceData, load_data)

---
## Load Config
---

In [2]:
# load yaml config
_config = load_config('config.yaml')
logging.basicConfig(level=logging.INFO)

---
## Human Data: Pilot
---

In [22]:
(data,
 sequence_data,
 meta_data,
 StimCode,
 RespCode
 ) = load_data(config=_config, experiment_class='human_pilot_experiment')

INFO:root:StimCode (284): [0. 1.]
INFO:root:RespCode (284): [1. 4.]


### Experiment Groupings : <subject, SessionType, SessionNumber>

In [23]:

def log_unique_session_types(StimCode):
    x = np.unique([k[1] for k in StimCode.keys()])
    print(x)

log_unique_session_types(StimCode)

['ams1' 'ams2' 'eds1' 'eds2' 'ids1' 'ids2' 'staircase1' 'staircase2'
 'training']


In [24]:
print('Human Pilot Data: ', meta_data.shape)
print('Session types: ', end=' ')
log_unique_session_types(StimCode)
StimCode


Human Pilot Data:  (284, 5)
Session types:  ['ams1' 'ams2' 'eds1' 'eds2' 'ids1' 'ids2' 'staircase1' 'staircase2'
 'training']


{(1,
  'training',
  0): array([1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1.,
        0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1.,
        0., 0., 1., 1., 1., 0., 0., 1., 0.], dtype=float32),
 (1,
  'training',
  1): array([0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
        0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
        0., 0., 1., 1., 0., 1., 0., 1., 0.], dtype=float32),
 (1,
  'ids1',
  0): array([0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
        0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        1., 0., 1., 1., 1., 1., 0., 1., 1.], dtype=float32),
 (1,
  'staircase1',
  0): array([0., 0., 0., 1., 1., 0., 0., 1.,

In [12]:
print(meta_data.head().to_markdown())

|    |   participantID | SessionType   |   SessionNum |   CondNum |   ReleMode |
|---:|----------------:|:--------------|-------------:|----------:|-----------:|
|  0 |               1 | training      |            1 |         1 |          1 |
|  1 |               1 | training      |            2 |         1 |          1 |
|  2 |               1 | ids1          |            1 |         2 |          1 |
|  3 |               1 | staircase1    |            1 |         3 |          1 |
|  4 |               1 | staircase1    |            2 |         3 |          1 |


In [6]:
log_sequence_data(sequence_data)

INFO:root:
        StimCode (len):     284
        RespCode (len):     284
        StimCode (keys):    dict_keys([(1, 'training', 0), (1, 'training', 1), (1, 'ids1', 0), (1, 'staircase1', 0), (1, 'staircase1', 1), (1, 'ams1', 0), (1, 'ams1', 1), (1, 'eds1', 0), (1, 'ids2', 0), (1, 'ids2', 1), (1, 'staircase2', 0), (1, 'ams2', 0), (1, 'ams2', 1), (1, 'eds2', 0), (2, 'training', 0), (2, 'training', 1), (2, 'training', 2), (2, 'training', 3), (2, 'training', 4), (2, 'training', 5), (2, 'training', 6), (2, 'training', 7), (2, 'training', 8), (2, 'ids1', 0), (2, 'staircase1', 0), (2, 'staircase1', 1), (2, 'ams1', 0), (2, 'ams1', 1), (2, 'eds1', 0), (2, 'ids2', 0), (2, 'staircase2', 0), (2, 'ams2', 0), (2, 'ams2', 1), (2, 'eds2', 0), (3, 'training', 0), (3, 'ids1', 0), (3, 'staircase1', 0), (3, 'staircase1', 1), (3, 'ams1', 0), (3, 'ams1', 1), (3, 'eds1', 0), (3, 'ids2', 0), (3, 'ids2', 1), (3, 'staircase2', 0), (3, 'staircase2', 1), (3, 'ams2', 0), (3, 'ams2', 1), (3, 'eds2', 0), (4, 'train

({(1,
   'training',
   0): array([1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1.,
         0., 0., 1., 1., 1., 0., 0., 1., 0.], dtype=float32),
  (1,
   'training',
   1): array([0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
         0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
         0., 0., 1., 1., 0., 1., 0., 1., 0.], dtype=float32),
  (1,
   'ids1',
   0): array([0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
         1., 0., 1., 1., 1., 1., 0., 1., 1.], dtype=float32),
  (1,
   'staircase1',
   0): array([0., 0., 0.

---
## Rat Data
---

In [27]:
(data,
 sequence_data,
 meta_data,
 StimCode,
 RespCode
 ) = load_data(config=_config, experiment_class='rat_experiment')

INFO:root:StimCode (571): [0. 1.]
INFO:root:RespCode (571): [1. 2. 3. 4.]


In [28]:

log_unique_session_types(StimCode)

['EDS' 'EDS_BL_Easy' 'EDS_BL_Hard' 'IDS' 'IDS_BL_Easy' 'RE' 'Stair']


---
## Human Data: Motiviation
---

In [17]:
(data,
 sequence_data,
 meta_data,
 StimCode,
 RespCode
 ) = load_data(config=_config, experiment_class='human_motivation_experiment')

INFO:root:StimCode (245): [0. 1.]
INFO:root:RespCode (245): [1. 4.]


In [18]:

log_unique_session_types(StimCode)

['ams1' 'ams2' 'eds1' 'eds2' 'ids1' 'ids2' 'staircase1' 'staircase2'
 'training']


In [21]:
print('Human Motivation Data: ', meta_data.shape)
print('Session types: ', end=' ')
log_unique_session_types(StimCode)
StimCode


Human Motivation Data:  (245, 5)
Session types:  ['ams1' 'ams2' 'eds1' 'eds2' 'ids1' 'ids2' 'staircase1' 'staircase2'
 'training']


{(1,
  'training',
  0): array([1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
        0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        0., 0., 1., 1., 1., 0., 1., 1., 1.], dtype=float32),
 (1,
  'training',
  1): array([1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0.,
        1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 0., 0., 1., 1., 1., 0., 0., 0.], dtype=float32),
 (1,
  'training',
  2): array([1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0.,
        0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 1., 0., 0., 1., 0.], dtype=float32),
 (1,
  'ids1',
  0): array([1., 1., 1., 0., 0., 1., 1., 0., 1

In [20]:
meta_data

,participantID,SessionType,SessionNum,CondNum,ReleMode
0,1,training,1,1,1
1,1,training,2,1,1
2,1,training,3,1,1
3,1,ids1,1,2,1
4,1,staircase1,1,3,1
...,...,...,...,...,...
240,20,ids2,1,6,1
241,20,staircase2,1,7,1
242,20,ams2,1,8,1
243,20,ams2,2,8,1
